In [91]:
import pandas as pd
import numpy as np
import os

In [92]:
columnDict = {
              'Unnamed: 25':'State',
              'Slab.1' : 'Slab',
              'Stringer /Multi-Beam or Girder.1': 'MultiBeam',
              'Girder & Floorbeam System.1': 'GirderFloorBeamSystem',
              'Tee Beam.1': 'TeaBeam',
              'Box Beam or Girders (Multiple).1': 'BoxBeamMultiple',
              'Box Beam or Girders (Single or Spread).1': 'BoxBeamSingle',
              'Frame (Except Culverts).1': 'Frame',
              'Orthotropic.1': 'Orthotropic',
              'Truss-Deck.1': 'TrussDeck',
              'Truss-Thru.1': 'TrussThru',
              'Arch-Deck.1': 'ArchDeck',
              'Arch-Thru.1': 'ArchThru',
              'Suspension.1': 'Suspension',
              'Stayed Girder.1': 'StayedGirder',
              'Movable-Lift.1': 'MovableLift',
              'Movable-Bascule.1': 'MovableBascule',
              'Movable-Swing.1': 'MovableSwing',
              'Tunnel.1': 'Tunnel',
              'Culvert.1': 'Culvert',
              'Mixed Types.1': 'MixedTypes',
              'Segmental Box Girder.1': 'SegmentalBoxGirder',
              'Channel Beam.1': 'ChannelBeam',
              'Other.1': 'Other',              
            }

In [93]:
sheetNames = list(map(str, range(1992,2014)))
#sheetNames = sheetNames[20:22]

In [94]:
len(dataDict_total[['State']])

52

In [116]:
len(sheetNames)

22

In [109]:
column_names = ['State', 'Year', 'Slab', 'MultiBeam', 'GirderFloorBeamSystem', 'TeaBeam', 'BoxBeamMultiple', 'ArchDeck', 'Culvert', 
            'TrussThru', 'Frame', 'BoxBeamSingle', 'TrussDeck']

df_allYears = pd.DataFrame(columns=column_names)

for sheetName in sheetNames:
    dataDict = pd.read_excel('NBIData.xlsx', sheet_name = sheetName, header = 3, engine='openpyxl')
    
    # Extract the total counts
    dataDict_total = dataDict.iloc[0:52, 25:51]
    
    # Extract the deficient bridge counts
    dataDict_def = dataDict.iloc[168:220, 25:51]
    
    # Rename columns of the dataframe
    dataDict_def.rename(columns = columnDict, inplace = True)
    dataDict_total.rename(columns = columnDict, inplace = True)
    
    # Reset indices of the dataframe 
    dataDict_def = dataDict_def.reset_index(drop=True) 
    dataDict_total = dataDict_total.reset_index(drop=True)  
    
    # Convert values of dataDict_def to float
    df_def_temp = dataDict_def.iloc[:,1:].copy().astype(float)
    df_def_temp.dtypes
    
    # Convert values of dataDict_total to float
    df_total_temp = dataDict_total.iloc[:,1:].copy().astype(float)
    df_total_temp.dtypes
    
    # Compute deficiency ratio by dividing df_def by df_total
    df_defRatio = df_def_temp / df_total_temp
    
    # Replace NaNs by 0 to account for data points where total count is actually 0 in the original data set
    df_defRatio.fillna(0, inplace =True)
    
    # Add corresponding year to the dataframe rows
    year = [sheetName] * len(dataDict_total[['State']])       # len(dataDict_total[['State']]) = 52
    yearDict = pd.DataFrame(year, columns = ['Year'])
    df_ID = pd.concat([dataDict_total[['State']], yearDict], axis = 1)
    
    # Add corresponding states to the dataframe rows
    df_defRatio = pd.concat([df_ID, df_defRatio], axis = 1)
    
        
    # Select important variables from df_defRatio
    df_defRatio = df_defRatio[column_names]
        
    
    # Append this dataframe to the combined dataframe
    df_allYears = pd.concat([df_allYears, df_defRatio], axis = 0)

    # Add target grade from report card
    
    
    
df_allYears.reset_index(drop = True, inplace = True)
    

In [111]:
df_allYears.tail()

,State,Year,Slab,MultiBeam,GirderFloorBeamSystem,TeaBeam,BoxBeamMultiple,ArchDeck,Culvert,TrussThru,Frame,BoxBeamSingle,TrussDeck
1139,WASHINGTON,2013,0.331335,0.276044,0.544119,0.338060,0.438852,0.392112,0.204236,0.514098,0.168329,0.525696,0.851734
1140,WEST VIRGINIA,2013,0.463707,0.211753,0.404103,0.567596,0.214580,0.565835,0.529438,0.327008,0.282401,0.144358,0.226640
1141,WISCONSIN,2013,0.099642,0.148156,0.291008,0.268167,0.265487,0.376592,0.049099,0.700330,0.387879,0.000000,0.633531
1142,WYOMING,2013,0.295514,0.229977,0.460110,0.209414,0.361643,0.655923,0.074208,0.884684,0.413407,0.283124,0.258494
1143,PUERTO RICO,2013,0.762105,0.361738,0.551498,0.604508,0.431512,1.000000,0.377161,0.765314,0.503061,0.113042,0.604385


In [112]:
# np.nan, 0
df_allYears = df_allYears.replace(np.nan, 0)

In [113]:
df_allYears.isna().values.any()


False

In [114]:
df_allYears.shape

(1144, 13)

In [118]:
# Export the dataframe as a .csv file ready for modeling.
df_allYears.to_csv("NBI_processed.csv", index = False, encoding="utf-8")